<a href="https://colab.research.google.com/github/b3ndy/Pok-Battle-AI-/blob/main/Pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import copy
import math
import time

# --- Configuration ---
AI_SEARCH_DEPTH = 4 #Move decisions
TEAM_SIZE = 3

# --- Type Effectiveness
def get_type_effectiveness(attack_type, defend_type1, defend_type2=None):
    return 1.0

# --- Move Class ---
class Move:
    def __init__(self, name, move_type, category, power, accuracy, pp):
        self.name = name
        self.type = move_type
        self.category = category # "Physical", "Special", "Status"
        self.power = power
        self.accuracy = accuracy # 0.0 to 1.0
        self.max_pp = pp
        self.current_pp = pp

    def __str__(self):
        return f"{self.name} ({self.type}/{self.category}) PP: {self.current_pp}/{self.max_pp}"

# --- pokemon class ---
class Pokemon:
    def __init__(self, name, types, stats, moves):
        self.name = name
        self.types = types # List or tuple
        self.stats = { # Base stats
            "hp": stats["hp"],
            "attack": stats["attack"],
            "defense": stats["defense"],
            "special_attack": stats["special_attack"],
            "special_defense": stats["special_defense"],
            "speed": stats["speed"]
        }
        self.current_hp = stats["hp"]
        # Ensure moves are deep copied *per instance*
        self.moves = [copy.deepcopy(m) for m in moves]
        self.status_condition = None # e.g., "Burn", "Poison", "Paralyze", "Sleep", None(although this part was a bit of hassle to work on so ive just left this one out)

    def get_stat(self, stat_name):
        value = self.stats.get(stat_name, 0)
        # Apply status condition modifiers if any
        if stat_name == "attack" and self.status_condition == "Burn":
            return value // 2
        if stat_name == "speed" and self.status_condition == "Paralyze":
            return value // 4 # Adjust multiplier as desired
        return value

    def take_damage(self, damage, simulate=False):
        """Applies damage to the pokemon. Only prints if not simulating."""
        effective_damage = max(0, math.floor(damage)) # Ensure non-negative integer damage
        self.current_hp = max(0, self.current_hp - effective_damage)
        if not simulate: # Only print during the actual battle, not simulation
            print(f"{self.name} took {effective_damage} damage! Remaining HP: {self.current_hp}")

    def is_fainted(self):
        return self.current_hp <= 0

    def display_status(self):
        status = f"{self.name} | HP: {self.current_hp}/{self.stats['hp']}"
        if self.status_condition:
            status += f" | Status: {self.status_condition}"
        return status

    def __str__(self):
        return self.name

# --- Battle Class ---
class Battle:

    def __init__(self, team1_pool, team2_pool, player1_is_ai: bool, player2_is_ai: bool, game_mode_description: str):
        if len(team1_pool) < TEAM_SIZE or len(team2_pool) < TEAM_SIZE:
            raise ValueError(f"Both teams must have at least {TEAM_SIZE} Pokemon.")

        self.teams = [copy.deepcopy(team1_pool), copy.deepcopy(team2_pool)]
        random.shuffle(self.teams[0])
        random.shuffle(self.teams[1])
        self.active_indices = [0, 0]

        self.turn = 0
        self.player_is_ai = [player1_is_ai, player2_is_ai]
        self.game_mode_description = game_mode_description # For display

        self.ai_player_indices = []
        if player1_is_ai: self.ai_player_indices.append(0)
        if player2_is_ai: self.ai_player_indices.append(1)


    # Helper Methods
    def get_active_pokemon(self, player_index):
        """Gets the currently active Pokemon for the player."""
        active_idx = self.active_indices[player_index]
        if 0 <= active_idx < len(self.teams[player_index]):
            pokemon = self.teams[player_index][active_idx]
            if pokemon.is_fainted():
                for i, c in enumerate(self.teams[player_index]):
                    if not c.is_fainted():
                        self.active_indices[player_index] = i
                        return c
                return None # No conscious Pokemon left
            return pokemon
        return None

    def get_opponent_index(self, player_index):
        return 1 - player_index

    def get_opponent_active_pokemon(self, player_index):
        opponent_idx = self.get_opponent_index(player_index)
        return self.get_active_pokemon(opponent_idx)

    def get_available_switch_ins(self, player_index):
        available = []
        current_active_index = self.active_indices[player_index]
        for i, pokemon in enumerate(self.teams[player_index]):
            if not pokemon.is_fainted() and i != current_active_index:
                available.append(i)
        return available

    def check_game_over(self):
        for i in range(2):
            if all(pokemon.is_fainted() for pokemon in self.teams[i]):
                return True
        return False

    def get_winner_index(self):
        p0_lost = all(pokemon.is_fainted() for pokemon in self.teams[0])
        p1_lost = all(pokemon.is_fainted() for pokemon in self.teams[1])
        if p0_lost and p1_lost: return -1
        if p0_lost: return 1
        if p1_lost: return 0
        return -1

    def display_battle_state(self, show_team_status=False):
        print("-" * 30)
        print(f"Turn {self.turn + 1}")

        active_p1 = self.get_active_pokemon(0)
        active_p2 = self.get_active_pokemon(1)
        p1_type = "(AI)" if self.player_is_ai[0] else "(Human)"
        p2_type = "(AI)" if self.player_is_ai[1] else "(Human)"
        print(f"Player 1 {p1_type} Active: {active_p1.display_status() if active_p1 else 'NONE (Team Fainted)'}")
        print(f"Player 2 {p2_type} Active: {active_p2.display_status() if active_p2 else 'NONE (Team Fainted)'}")

        if show_team_status:
            print("-- Team Status --")
            for i in range(2):
                p_type = "(AI)" if self.player_is_ai[i] else "(Human)"
                team_status = []
                for idx, pokemon in enumerate(self.teams[i]):
                    status_char = "✓" if not pokemon.is_fainted() else "X"
                    active_char = "*" if idx == self.active_indices[i] and not pokemon.is_fainted() else ""
                    team_status.append(f"{active_char}{pokemon.name} ({status_char})")
                print(f"  Player {i+1} {p_type}: [{', '.join(team_status)}]")
        print("-" * 30)

    def calculate_damage(self, attacker_idx, defender_idx, move, simulate=False):
        attacker = self.get_active_pokemon(attacker_idx)
        defender = self.get_active_pokemon(defender_idx)

        if not attacker or not defender or defender.is_fainted(): return 0
        if move.power <= 0 or move.category == "Status": return 0

        LEVEL = 50
        if move.category == "Physical":
            attack_stat = attacker.get_stat("attack")
            defense_stat = defender.get_stat("defense")
        elif move.category == "Special":
            attack_stat = attacker.get_stat("special_attack")
            defense_stat = defender.get_stat("special_defense")
        else:
            return 0

        if defense_stat <= 0: defense_stat = 1
        damage = (((2 * LEVEL / 5 + 2) * move.power * (attack_stat / defense_stat)) / 50) + 2
        damage *= random.uniform(0.85, 1.0)
        if damage < 1: damage = 1
        return math.floor(damage)

    def apply_status_effect(self, status, target_idx, simulate=False):
        target = self.get_active_pokemon(target_idx)
        if target and target.status_condition is None:
            target.status_condition = status
            if not simulate:
                print(f"{target.name} was inflicted with {status}!")

    def handle_status_damage(self, player_index, simulate=False):
        pokemon = self.get_active_pokemon(player_index)
        if pokemon and not pokemon.is_fainted() and pokemon.status_condition in ["Burn", "Poison"]:
            damage = max(1, math.floor(pokemon.stats['hp'] / 16))
            if not simulate:
                 print(f"{pokemon.name} is hurt by {pokemon.status_condition}!")
            pokemon.take_damage(damage, simulate=simulate)
            return True
        return False

    def handle_pre_turn_status(self, player_index, simulate=False):
        pokemon = self.get_active_pokemon(player_index)
        if not pokemon or pokemon.is_fainted(): return False

        if pokemon.status_condition == "Paralyze":
            if random.random() < 0.25:
                if not simulate: print(f"{pokemon.name} is paralyzed! It can't move!")
                return True
        # Add Sleep/Freeze checks here
        if pokemon.status_condition == "Sleep": # Basic sleep: 50% chance to wake each turn, or auto-wake after 3 turns.
            # This needs a turn counter for sleep if you want fixed duration.
            # For simplicity, let's do a 50% wake-up chance.
            if hasattr(pokemon, 'sleep_turns') and pokemon.sleep_turns > 0:
                pokemon.sleep_turns -=1
                if not simulate: print(f"{pokemon.name} is fast asleep. ({pokemon.sleep_turns} turns left)")
                return True # Still sleeping
            else: # Chance to wake or if turns ran out
                if random.random() < 0.5 or not hasattr(pokemon, 'sleep_turns'): # 50% wake, or if no sleep_turns (first turn of sleep)
                    if not simulate: print(f"{pokemon.name} woke up!")
                    pokemon.status_condition = None
                    if hasattr(pokemon, 'sleep_turns'): delattr(pokemon, 'sleep_turns')
                    return False # Woke up
                else: # Failed to wake up
                    if not hasattr(pokemon, 'sleep_turns'): pokemon.sleep_turns = random.randint(1,3) # Set initial duration if just fell asleep
                    if not simulate: print(f"{pokemon.name} is fast asleep.")
                    return True
        return False

    def execute_turn(self, player_index, move_index, simulate=False):
        attacker = self.get_active_pokemon(player_index)
        defender = self.get_opponent_active_pokemon(player_index)
        opponent_index = self.get_opponent_index(player_index)

        if not attacker or attacker.is_fainted(): return False
        if not defender: return False

        if move_index < 0 or move_index >= len(attacker.moves):
            if not simulate: print("Invalid move index.")
            return False

        move = attacker.moves[move_index]
        if move.current_pp <= 0:
            if not simulate: print(f"{attacker.name} tried to use {move.name}, but has no PP left!")
            return True

        move.current_pp -= 1
        if not simulate: print(f"{attacker.name} used {move.name}!")

        if random.random() > move.accuracy:
            if not simulate: print("But it missed!")
            return True

        if move.category == "Status":
            if move.name == "Thunder Wave":
                 if defender and "Electric" not in defender.types and defender.status_condition is None:
                     self.apply_status_effect("Paralyze", opponent_index, simulate=simulate)
                 elif not simulate:
                     print(f"It doesn't affect {defender.name if defender else 'target'}...")
            elif move.name == "Hypnosis" or move.name == "Sleep Powder":
                 if defender and defender.status_condition is None:
                     self.apply_status_effect("Sleep", opponent_index, simulate=simulate)
                     if defender: defender.sleep_turns = random.randint(1, 3) # Set sleep duration
                 elif not simulate:
                      print(f"It doesn't affect {defender.name if defender else 'target'}...")
            elif not simulate:
                print(f"Status move '{move.name}' effect not fully implemented.")
        else:
             damage = self.calculate_damage(player_index, opponent_index, move, simulate=simulate)
             if defender and not defender.is_fainted():
                 defender.take_damage(damage, simulate=simulate)
                 if move.name == "Flamethrower" and random.random() < 0.1:
                      if defender.status_condition is None and "Fire" not in defender.types:
                          self.apply_status_effect("Burn", opponent_index, simulate=simulate)
             elif not simulate and defender:
                 print(f"{defender.name} has already fainted!")
        return True

    def get_possible_moves(self, player_index):
        active_pokemon = self.get_active_pokemon(player_index)
        if not active_pokemon or active_pokemon.is_fainted():
            return []
        return [i for i, move in enumerate(active_pokemon.moves) if move.current_pp > 0]

    def choose_human_switch(self, player_index):
        available_indices = self.get_available_switch_ins(player_index)
        if not available_indices: return -1

        fainted_pokemon = self.teams[player_index][self.active_indices[player_index]]
        print(f"\n{fainted_pokemon.name} fainted!")
        print(f"Player {player_index + 1}, choose your next Pokémon:")
        for i, switch_idx in enumerate(available_indices):
            pokemon = self.teams[player_index][switch_idx]
            print(f"{i + 1}: {pokemon.name} ({pokemon.current_hp}/{pokemon.stats['hp']})")

        while True:
            try:
                choice = input(f"Enter number (1-{len(available_indices)}): ")
                choice_int = int(choice) - 1
                if 0 <= choice_int < len(available_indices):
                    chosen_index = available_indices[choice_int]
                    print(f"Go, {self.teams[player_index][chosen_index].name}!")
                    return chosen_index
                else:
                    print("Invalid choice.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except EOFError: exit()

    def choose_ai_switch(self, player_index):
        available_indices = self.get_available_switch_ins(player_index)
        if not available_indices: return -1

        print(f"AI Player {player_index + 1} is choosing a switch-in...")
        time.sleep(0.5)

        best_choice = -1
        max_hp = -1
        for idx in available_indices:
            pokemon = self.teams[player_index][idx]
            if pokemon.current_hp > max_hp:
                max_hp = pokemon.current_hp
                best_choice = idx
        if best_choice == -1: best_choice = random.choice(available_indices)

        switch_in_pokemon = self.teams[player_index][best_choice]
        print(f"AI Player {player_index + 1} sends out {switch_in_pokemon.name}!")
        return best_choice

    def force_switch(self, player_index):
        available = self.get_available_switch_ins(player_index)
        if not available: return False

        if self.player_is_ai[player_index]:
            new_active_index = self.choose_ai_switch(player_index)
        else:
            new_active_index = self.choose_human_switch(player_index)

        if new_active_index != -1:
            self.active_indices[player_index] = new_active_index
            return True
        else:
            if available:
                 self.active_indices[player_index] = available[0]
                 print(f"Fallback: Automatically switched to {self.teams[player_index][available[0]].name}")
                 return True
            return False

    def evaluate_state(self, ai_eval_player_index):
        opponent_index = self.get_opponent_index(ai_eval_player_index)
        if self.check_game_over():
            winner = self.get_winner_index()
            if winner == ai_eval_player_index: return float('inf')
            if winner == opponent_index: return -float('inf')
            return 0

        ai_team = self.teams[ai_eval_player_index]
        opp_team = self.teams[opponent_index]
        ai_total_hp_ratio = sum(c.current_hp / c.stats['hp'] for c in ai_team if not c.is_fainted())
        opp_total_hp_ratio = sum(c.current_hp / c.stats['hp'] for c in opp_team if not c.is_fainted())
        ai_conscious_count = sum(1 for c in ai_team if not c.is_fainted())
        opp_conscious_count = sum(1 for c in opp_team if not c.is_fainted())

        score = (ai_conscious_count - opp_conscious_count) * 1.0
        score += (ai_total_hp_ratio - opp_total_hp_ratio) * 0.5

        ai_active = self.get_active_pokemon(ai_eval_player_index)
        opp_active = self.get_active_pokemon(opponent_index)
        status_modifier = 0
        if ai_active and ai_active.status_condition in ["Burn", "Poison", "Paralyze", "Sleep"]:
            status_modifier -= 0.1
        if opp_active and opp_active.status_condition in ["Burn", "Poison", "Paralyze", "Sleep"]:
            status_modifier += 0.1
        score += status_modifier
        return score


    #actual AI stuff
    def minimax(self, depth, maximizing_player_index, current_turn_player_index, alpha, beta):
        opponent_index = self.get_opponent_index(current_turn_player_index)
        if depth == 0 or self.check_game_over():
            return self.evaluate_state(maximizing_player_index), -1

        current_active = self.get_active_pokemon(current_turn_player_index)
        if not current_active or current_active.is_fainted():
             return self.evaluate_state(maximizing_player_index), -1

        possible_moves = self.get_possible_moves(current_turn_player_index)
        if not possible_moves:
             return self.evaluate_state(maximizing_player_index), -1

        best_move_index = possible_moves[0]
        if current_turn_player_index == maximizing_player_index:
            max_eval = -float('inf')
            for move_idx in possible_moves:
                temp_battle_state = copy.deepcopy(self)
                temp_battle_state.execute_turn(current_turn_player_index, move_idx, simulate=True)
                active_after_move = temp_battle_state.get_active_pokemon(current_turn_player_index)
                opponent_active_after_move = temp_battle_state.get_active_pokemon(opponent_index)
                if active_after_move: temp_battle_state.handle_status_damage(current_turn_player_index, simulate=True)
                if opponent_active_after_move: temp_battle_state.handle_status_damage(opponent_index, simulate=True)
                evaluation, _ = temp_battle_state.minimax(depth - 1, maximizing_player_index, opponent_index, alpha, beta)
                if evaluation > max_eval:
                    max_eval = evaluation
                    best_move_index = move_idx
                alpha = max(alpha, evaluation)
                if beta <= alpha: break
            return max_eval, best_move_index
        else:
            min_eval = float('inf')
            for move_idx in possible_moves:
                temp_battle_state = copy.deepcopy(self)
                temp_battle_state.execute_turn(current_turn_player_index, move_idx, simulate=True)
                active_after_move = temp_battle_state.get_active_pokemon(current_turn_player_index)
                opponent_active_after_move = temp_battle_state.get_active_pokemon(opponent_index)
                if active_after_move: temp_battle_state.handle_status_damage(current_turn_player_index, simulate=True)
                if opponent_active_after_move: temp_battle_state.handle_status_damage(opponent_index, simulate=True)
                evaluation, _ = temp_battle_state.minimax(depth - 1, maximizing_player_index, opponent_index, alpha, beta)
                if evaluation < min_eval:
                    min_eval = evaluation
                    best_move_index = move_idx
                beta = min(beta, evaluation)
                if beta <= alpha: break
            return min_eval, best_move_index

    def get_ai_move(self, player_index):
        active_pokemon = self.get_active_pokemon(player_index)
        if not active_pokemon or active_pokemon.is_fainted(): return -1

        print(f"{active_pokemon.name} (AI Player {player_index + 1}) is thinking...")
        start_time = time.time()
        _, best_move_index = self.minimax(AI_SEARCH_DEPTH, player_index, player_index, -float('inf'), float('inf'))
        end_time = time.time()
        print(f"(Thinking time: {end_time - start_time:.2f}s)")

        possible_moves = self.get_possible_moves(player_index)
        if not possible_moves: return -1
        if best_move_index == -1 or best_move_index not in possible_moves:
             print("AI fallback: Choosing first available move.")
             return possible_moves[0]
        return best_move_index

    def get_human_move(self, player_index):
        player_pokemon = self.get_active_pokemon(player_index)
        if not player_pokemon or player_pokemon.is_fainted(): return -1

        print(f"\nPlayer {player_index + 1} ({player_pokemon.name}), choose your move:")
        possible_moves = self.get_possible_moves(player_index)
        if not possible_moves:
            print("No moves available!")
            return -1

        for i, move_idx in enumerate(possible_moves):
            print(f"{i + 1}: {player_pokemon.moves[move_idx]}")

        while True:
            try:
                choice = input(f"Enter move number (1-{len(possible_moves)}): ")
                choice_int = int(choice) - 1
                if 0 <= choice_int < len(possible_moves):
                    return possible_moves[choice_int]
                else:
                    print("Invalid choice.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except EOFError: exit()

    def run_battle(self):
        print("\nBattle Start! 3v3 Teams")
        print(f"Player 1 Team: {[c.name for c in self.teams[0]]}")
        print(f"Player 2 Team: {[c.name for c in self.teams[1]]}")
        print(f"Mode: {self.game_mode_description}") # Use the description

        while not self.check_game_over():
            self.turn += 1
            self.display_battle_state(show_team_status=(self.turn==1))

            # Pre-turn safety switch if needed
            for i in range(2):
                active_c = self.get_active_pokemon(i)
                if not active_c or active_c.is_fainted():
                    print(f"Player {i+1}'s active Pokémon fainted before turn start. Forcing switch...")
                    if not self.force_switch(i):
                        if self.check_game_over(): break # Game ended due to no switch
            if self.check_game_over(): break


            active_p1 = self.get_active_pokemon(0)
            active_p2 = self.get_active_pokemon(1)
            if not active_p1 or not active_p2: # Should be caught by game_over if one team is all out
                print("Error: An active Pokémon is missing.")
                break

            p1_speed = active_p1.get_stat("speed")
            p2_speed = active_p2.get_stat("speed")
            order = [0, 1] if p1_speed > p2_speed else [1, 0] if p2_speed > p1_speed else random.sample([0, 1], 2)
            print(f"{self.get_active_pokemon(order[0]).name} goes first this turn.")

            moves_chosen = {}
            can_move = {0: True, 1: True}

            for player_idx in order:
                 current_active = self.get_active_pokemon(player_idx)
                 if not current_active or current_active.is_fainted():
                      can_move[player_idx] = False
                      continue

                 if self.handle_pre_turn_status(player_idx, simulate=False):
                      can_move[player_idx] = False
                      moves_chosen[player_idx] = -1
                      continue

                 is_ai = self.player_is_ai[player_idx]
                 if is_ai:
                     move_idx = self.get_ai_move(player_idx)
                 else:
                     move_idx = self.get_human_move(player_idx)

                 if move_idx == -1:
                      print(f"{current_active.name} has no moves to use!")
                      can_move[player_idx] = False
                 moves_chosen[player_idx] = move_idx

            for player_idx in order:
                if self.check_game_over(): break
                opponent_idx = self.get_opponent_index(player_idx)
                active_pokemon = self.get_active_pokemon(player_idx)

                if not active_pokemon or active_pokemon.is_fainted() or not can_move[player_idx]:
                    continue

                move_idx = moves_chosen.get(player_idx, -1)
                if move_idx == -1:
                    print(f"{active_pokemon.name} couldn't move!")
                    continue

                self.execute_turn(player_idx, move_idx, simulate=False)
                opponent_active = self.get_active_pokemon(opponent_idx)

                if opponent_active and opponent_active.is_fainted():
                    if self.check_game_over(): break
                    if not self.force_switch(opponent_idx): break

                if active_pokemon.is_fainted(): # Check if attacker fainted
                    if self.check_game_over(): break
                    if not self.force_switch(player_idx): break
            if self.check_game_over(): break

            print("\n--- End of Turn Status Effects ---")
            for player_idx in order:
                if self.check_game_over(): break
                pokemon = self.get_active_pokemon(player_idx)
                if pokemon and not pokemon.is_fainted():
                    damaged_by_status = self.handle_status_damage(player_idx, simulate=False)
                    if damaged_by_status and pokemon.is_fainted():
                        print(f"\n{pokemon.name} fainted from its status condition!")
                        if self.check_game_over(): break
                        if not self.force_switch(player_idx): break
            if self.check_game_over(): break

            try:
                # Pause if at least one human player is present
                if not (self.player_is_ai[0] and self.player_is_ai[1]):
                    input("\nPress Enter to continue...")
                else:
                    print("\nContinuing...")
                    time.sleep(1)
            except EOFError: break

        print("\n" + "="*15 + " GAME OVER " + "="*15)
        self.display_battle_state(show_team_status=True)
        winner_index = self.get_winner_index()
        if winner_index == 0: print("Player 1 wins!")
        elif winner_index == 1: print("Player 2 wins!")
        else: print("It's a draw or an unexpected end state!")


# --- Define pokemons and Moves ---
flamethrower = Move("Flamethrower", "Fire", "Special", 90, 1.0, 15)
dragon_claw = Move("Dragon Claw", "Dragon", "Physical", 80, 1.0, 15)
air_slash = Move("Air Slash", "Flying", "Special", 75, 0.95, 15)
thunder_wave = Move("Thunder Wave", "Electric", "Status", 0, 0.9, 20)
hydro_pump = Move("Hydro Pump", "Water", "Special", 110, 0.8, 5)
ice_beam = Move("Ice Beam", "Ice", "Special", 90, 1.0, 10)
skull_bash = Move("Skull Bash", "Normal", "Physical", 130, 1.0, 10)
bite = Move("Bite", "Dark", "Physical", 60, 1.0, 25)
vine_whip = Move("Vine Whip", "Grass", "Physical", 45, 1.0, 25)
sludge_bomb = Move("Sludge Bomb", "Poison", "Special", 90, 1.0, 10)
sleep_powder = Move("Sleep Powder", "Grass", "Status", 0, 0.75, 15)
tackle = Move("Tackle", "Normal", "Physical", 40, 1.0, 35)
thunderbolt = Move("Thunderbolt", "Electric", "Special", 90, 1.0, 15)
quick_attack = Move("Quick Attack", "Normal", "Physical", 40, 1.0, 30)
shadow_ball = Move("Shadow Ball", "Ghost", "Special", 80, 1.0, 15)
hypnosis = Move("Hypnosis", "Psychic", "Status", 0, 0.6, 20)
flare_blitz = Move("Flare Blitz", "Fire", "Physical", 120, 1.0, 15)
extreme_speed = Move("Extreme Speed", "Normal", "Physical", 80, 1.0, 5)

charizard_stats = {"hp": 78, "attack": 84, "defense": 78, "special_attack": 109, "special_defense": 85, "speed": 100}
charizard_moves = [flamethrower, dragon_claw, air_slash, thunder_wave]
charizard = Pokemon("Charizard", ["Fire", "Flying"], charizard_stats, charizard_moves)

blastoise_stats = {"hp": 79, "attack": 83, "defense": 100, "special_attack": 85, "special_defense": 105, "speed": 78}
blastoise_moves = [hydro_pump, ice_beam, skull_bash, bite]
blastoise = Pokemon("Blastoise", ["Water"], blastoise_stats, blastoise_moves)

venusaur_stats = {"hp": 80, "attack": 82, "defense": 83, "special_attack": 100, "special_defense": 100, "speed":80}
venusaur_moves = [vine_whip, sludge_bomb, sleep_powder, tackle]
venusaur = Pokemon("Venusaur", ["Grass", "Poison"], venusaur_stats, venusaur_moves)

pikachu_stats = {"hp": 35, "attack": 40, "defense": 40, "special_attack": 60, "special_defense": 50, "speed": 120}
pikachu_moves = [thunderbolt, quick_attack, thunder_wave, tackle]
pikachu = Pokemon("Pikachu", ["Electric"], pikachu_stats, pikachu_moves)

gengar_stats = {"hp": 60, "attack": 65, "defense": 60, "special_attack": 130, "special_defense": 75, "speed": 110}
gengar_moves = [shadow_ball, sludge_bomb, thunderbolt, hypnosis]
gengar = Pokemon("Gengar", ["Ghost", "Poison"], gengar_stats, gengar_moves)

arcanine_stats = {"hp": 90, "attack": 110, "defense": 80, "special_attack": 100, "special_defense": 80, "speed": 95}
arcanine_moves = [flare_blitz, extreme_speed, flamethrower, bite]
arcanine = Pokemon("Arcanine", ["Fire"], arcanine_stats, arcanine_moves)


# --- Start Game ---
if __name__ == "__main__":
    print("Welcome to PokéBattle AI!")
    print("Choose your game setup:")
    print("1: Play against an AI opponent")
    print("2: Watch an AI vs AI simulation")
    main_choice = ""
    while main_choice not in ['1', '2']:
        main_choice = input("Enter choice (1-2): ")

    p1_is_ai_flag = False
    p2_is_ai_flag = False
    game_mode_desc = ""

    if main_choice == '1':
        player_choice = ""
        while player_choice not in ['1', '2']:
            player_choice = input("Do you want to be Player 1 or Player 2? (Enter 1 or 2): ")
        if player_choice == '1':
            p1_is_ai_flag = False
            p2_is_ai_flag = True
            game_mode_desc = "Human (P1) vs AI (P2)"
        else: # player_choice == '2'
            p1_is_ai_flag = True
            p2_is_ai_flag = False
            game_mode_desc = "AI (P1) vs Human (P2)"
    elif main_choice == '2':
        p1_is_ai_flag = True
        p2_is_ai_flag = True
        game_mode_desc = "AI (P1) vs AI (P2)"

    all_defined_pokemons = [
        charizard, blastoise, venusaur, pikachu, gengar, arcanine
    ]
    required_pokemons = TEAM_SIZE * 2
    if len(all_defined_pokemons) < required_pokemons:
        print(f"Error: Need at least {required_pokemons} unique pokemons defined.")
        exit()

    shuffled_pool = random.sample(all_defined_pokemons, len(all_defined_pokemons))
    team1_pool = [copy.deepcopy(c) for c in shuffled_pool[:TEAM_SIZE]]
    team2_pool = [copy.deepcopy(c) for c in shuffled_pool[TEAM_SIZE : required_pokemons]]

    print("\n--- Teams Assigned ---")
    print(f"Player 1 Team Pool: {[c.name for c in team1_pool]}")
    print(f"Player 2 Team Pool: {[c.name for c in team2_pool]}")
    print("(Team member order will be randomized again at battle start by the Battle class)")

    battle = Battle(team1_pool, team2_pool,
                    player1_is_ai=p1_is_ai_flag,
                    player2_is_ai=p2_is_ai_flag,
                    game_mode_description=game_mode_desc)
    battle.run_battle()

Welcome to PokéBattle AI!
Choose your game setup:
1: Play against an AI opponent
2: Watch an AI vs AI simulation
Enter choice (1-2): 2

--- Teams Assigned ---
Player 1 Team Pool: ['Pikachu', 'Venusaur', 'Blastoise']
Player 2 Team Pool: ['Gengar', 'Arcanine', 'Charizard']
(Team member order will be randomized again at battle start by the Battle class)

Battle Start! 3v3 Teams
Player 1 Team: ['Pikachu', 'Venusaur', 'Blastoise']
Player 2 Team: ['Charizard', 'Gengar', 'Arcanine']
Mode: AI (P1) vs AI (P2)
------------------------------
Turn 2
Player 1 (AI) Active: Pikachu | HP: 35/35
Player 2 (AI) Active: Charizard | HP: 78/78
-- Team Status --
  Player 1 (AI): [*Pikachu (✓), Venusaur (✓), Blastoise (✓)]
  Player 2 (AI): [*Charizard (✓), Gengar (✓), Arcanine (✓)]
------------------------------
Pikachu goes first this turn.
Pikachu (AI Player 1) is thinking...
(Thinking time: 0.03s)
Charizard (AI Player 2) is thinking...
(Thinking time: 0.05s)
Pikachu used Thunderbolt!
Charizard took 26 dama